In [1]:
## Get libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, QuantileRegressor
from sklearn.metrics import root_mean_squared_error
import pickle
import mlflow
import statsmodels.api as sm
from scipy import stats
import plotly.express as px
from darts import TimeSeries
from darts.models import FFT, ExponentialSmoothing
import catboost as cb
import datetime as dt
import math
from src.seasonal import MultiSeasonalDecomposition # STL, FourierDecomposition,
from statsforecast import StatsForecast
from statsforecast.models import MSTL, AutoARIMA, AutoETS, Naive
from statsmodels.tsa.seasonal import seasonal_decompose, STL

import plotly.graph_objects as go
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'sklearn.ensemble._hist_gradient_boosting._gradient_boosting'

In [ ]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
# inputs
exp_no = 13
exp_desc = 'Catboost model + Mul Seas + MSTL for month'

target_col = 'Temperature'
tracking_col = ["id", 'date']

windows_cv_num_periods = 6 # Split the data into 10 parts and perform windows based TS cross validation and 
                            # Out of sample TS-prediction. (These TS-predictions will go as input to final model)
evaluation_start_date = '2018-04-04'
time_feature_origin = '2016-01-01 00:00:00'

# Coverage - Represents prediction interval expected coverage in %.
quantiles = [i * 0.10 for i in range(1, 10)] # probabilities for which we should predict.
quantiles.insert(0, 0.05)
quantiles = [round(q, 3) for q in quantiles]
quantiles

In [ ]:
new_quantiles = []
for quantile in quantiles:
    
    lb = round(quantile/2, 3)
    ub = 1 - lb
    lb = str(lb)
    ub = str(ub) #'%.3f' % ub
    new_quantiles.append(lb)
    new_quantiles.append(ub)

new_quantiles.insert(0, '0.5')
new_quantiles.sort()

In [ ]:
print(new_quantiles)

In [ ]:
## Get data 
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
sample_df = pd.read_csv('data/sample_submission.csv')

In [ ]:
# cyclic encoding 

def sin_cos_encoding(column, column_max = None):
    """
    Encode a pandas DataFrame column into sine and cosine values.

    Parameters:
    - column: pandas Series
        The input column to be encoded.

    Returns:
    - sin_values: list
        List of sine values for each element in the input column.
    - cos_values: list
        List of cosine values for each element in the input column.

    This function takes a pandas DataFrame column, computes the sine and cosine
    values for each element in the column, and returns two lists containing
    these encoded values. The encoding is useful for converting cyclic data,
    such as angles or time, into a format that can be more easily processed
    by machine learning models.
    """
    if column_max is None:
        max_value = column.max()
    else:
        max_value = column_max
    
    sin_values = [math.sin((2*math.pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*math.pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values


# Define the function to transform and rename columns
def cyclic_feature_transformation(df, columns, column_max = None):

    for index, column_name in enumerate(columns):
        
        if column_max is None:
            i_column_max = None
        else:
            i_column_max = column_max[index]
            
        # Call cyclic_feature_transformation function
        sin_values, cos_values = sin_cos_encoding(df[column_name], i_column_max)
        
        # Rename the columns
        new_column_name_sin = f'{column_name}_sin'
        new_column_name_cos = f'{column_name}_cos'
        # df.rename(columns={column_name: new_column_name_sin, 
        #                    new_column_name_sin: new_column_name_cos}, inplace=True)
        
        # Assign new values to the renamed columns
        df[new_column_name_sin] = sin_values
        df[new_column_name_cos] = cos_values
        
        # Drop the original column
        df.drop(column_name, axis=1, inplace=True)

In [ ]:
dt_obj = dt.datetime.strptime(time_feature_origin,  '%Y-%m-%d %H:%M:%S')

# add time dependant variable.
train_df['date'] = pd.to_datetime(train_df['date'])
# train_df['new_date'] = pd.to_datetime(train_df['date'])
# train_df['day_of_week'] = train_df.date.dt.dayofweek
train_df['day_of_year'] = train_df.date.dt.day_of_year
train_df['hour'] = train_df.date.dt.hour
train_df['day_of_month'] = train_df.date.dt.day
# train_df['week_of_year'] = train_df.date.dt.isocalendar().week
# train_df['month'] = train_df.date.dt.month
# train_df['quarter'] = train_df.date.dt.quarter
# train_df['year'] = train_df.date.dt.year
# train_df.set_index("new_date", inplace=True)
# train_df['date_int'] = (train_df['date'] - dt_obj) / np.timedelta64(1, 'h')

# cyclic_feature_transformation(train_df, ['day_of_week', 'day_of_year', 'month'], [7, 366, 12])
cyclic_feature_transformation(train_df, ['hour', 'day_of_year', 'day_of_month'], [24, 366, 31])

test_df['date'] = pd.to_datetime(test_df['date'])
# test_df['new_date'] = pd.to_datetime(test_df['date'])
# test_df['day_of_week'] = test_df.date.dt.dayofweek
test_df['day_of_year'] = test_df.date.dt.day_of_year
test_df['hour'] = test_df.date.dt.hour
test_df['day_of_month'] = test_df.date.dt.day
# test_df['week_of_year'] = test_df.date.dt.isocalendar().week
# test_df['month'] = test_df.date.dt.month
# test_df['quarter'] = test_df.date.dt.quarter
# test_df['year'] = test_df.date.dt.year
# test_df.set_index("new_date", inplace=True)
# test_df['date_int'] = (test_df['date'] - dt_obj) / np.timedelta64(1, 'h')

# cyclic_feature_transformation(test_df, ['day_of_week', 'day_of_year', 'month'], [7, 366, 12])
cyclic_feature_transformation(test_df, ['hour', 'day_of_year', 'day_of_month'], [24, 366, 31])

train_df.head(6)

### Required functions

In [ ]:
### Carl McBride's functions.

def crps(submission, solution):
    """
    This routine returns the mean continuous ranked probability score (CRPS).
    Each individual CRPS score is numerically integrated using 23 points.
    The extremal points (100% coverage) are competition fixed at -30 and 60.
    The "submission" dataframe: the last 21 columns should be the predictions
    The "solution" dataframe must contain a "Temperature" column (the "ground truth")
    
    Author: Carl McBride Ellis
    Version: 1.0.0
    Date: 2024-03-30
    """
        
    # A list of the requested quantile values, along with added 100% coverage endpoints 
    # (these values are all competition fixed)
    # the 0.5 quantile is the "zero coverage" forecast i.e. the median point prediction
    quantiles = [0.00, 0.025, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95, 0.975, 1.00]
    submission_tmp = submission.copy()
    # inset the y_true values to the submission_tmp dataframe to the LHS
    submission_tmp.insert(0, "Temperature", solution["Temperature"].values)
    
    CRPS = 0
    for index, row in submission_tmp.iterrows():
        x_values = row[-(len(quantiles)-2):] # column name agnostic
        y_true = row["Temperature"] # the ground truth value
        
        x_values = [float(i) for i in x_values] # make sure all x values are floats
        # add extremal 100% quantile x-values so as to be sure to bracket all possible y_true values
        # note: any changing of these values will change the score
        x_values.append(-30.0)
        x_values.append( 60.0)
        x_values.sort() # sort x values into ascending order (no quantile crossing)

        # split predictions to the left and right of the true value
        # get items below the true value (y_true)
        LHS_keys = [i for i,x in enumerate(x_values) if x < y_true]
        # get items above the true value (y_true)
        RHS_keys = [i for i,x in enumerate(x_values) if x >= y_true]

        # quantiles and predictions below the true value (y_true)
        LHS_values = [x_values[i] for i in LHS_keys]
        LHS_quantiles = [quantiles[i] for i in LHS_keys]

        # quantiles and predictions above the true value (y_true)
        RHS_values = [x_values[i] for i in RHS_keys]
        RHS_quantiles = [quantiles[i] for i in RHS_keys]

        # also calculate quantile at y (q_at_y_true)
        x1, y1 = LHS_values[-1], LHS_quantiles[-1]
        x2, y2 = RHS_values[0], RHS_quantiles[0]
        q_at_y_true = ((y2-y1)*(y_true-x1)/(x2-x1))+y1

        # add y_true and q_at_y_true to RHS of LHS list
        LHS_values.append(y_true)
        LHS_quantiles.append(q_at_y_true)

        # add y_true and q_at_y_true to LHS of RHS list
        RHS_values.insert(0, y_true)
        RHS_quantiles.insert(0, q_at_y_true)

        # integrate the LHS as a sum of trapezium for CDF**2
        LHS_integral = 0
        for i in range(len(LHS_values)-1):
            LHS_integral += (0.5 * (LHS_values[i+1]-LHS_values[i]) * (LHS_quantiles[i]**2 + LHS_quantiles[i+1]**2) )

        # integrate the RHS as a sum of trapezium for (1-CDF)**2
        RHS_integral = 0
        for i in range(len(RHS_values)-1):
            RHS_integral += (0.5 * (RHS_values[i+1]-RHS_values[i]) * ((1-RHS_quantiles[i])**2 +(1-RHS_quantiles[i+1])**2 ) )

        CRPS += (LHS_integral + RHS_integral)

    del submission_tmp
    # calculate the mean CRPS
    CRPS = CRPS/len(submission)
    return CRPS


def coverage_report(submission, solution):
    """
    Version: 1.0.1
    """
    y_true = solution["Temperature"].values
    # this does not take the "zero coverage" prediction into account
    # which is assumed to be located in submission.csv column -11
    coverages = [95, 90, 80, 70, 60, 50, 40, 30, 20, 10]
    N = len(coverages)
    # ANSI color codes
    BOLD_RED = '\033[1;31m'
    BOLD_GREEN = '\033[1;32m'
    END_COLOR = '\033[0m'
    
    def mean_coverage(y_pred_low,y_true,y_pred_up):
        return ( (y_pred_low <= y_true) & (y_pred_up >= y_true) ).mean()
    
    for i, coverage in enumerate(coverages):
        lower_col, upper_col = (2*N+1-i), (i+1)
        actual_coverage = mean_coverage(submission.iloc[:,-lower_col], y_true, submission.iloc[:,-upper_col])
        actual_coverage = round(actual_coverage*100,2)
        if actual_coverage >= coverages[i]:
            print(BOLD_GREEN, "Ideal: {}% Actual: {}% [PASS]".format(coverage, actual_coverage), END_COLOR)
        else:
            print(BOLD_RED, "Ideal: {}% Actual: {}% [FAIL]".format(coverage, actual_coverage), END_COLOR)

In [ ]:
def ConvertDataFrameToArrays(train_df, test_df, target_col, tracking_col):
    ## Convert data to model consumable format.

    # Extract features (X) and target variable (y) from the DataFrame    
    cols_to_rem = [target_col] + tracking_col
    X_train = train_df.drop(columns=cols_to_rem)  
    y_train = train_df[target_col]
    X_test = test_df.drop(columns = tracking_col)  

    # Convert DataFrame to numpy arrays
    X_train_array = X_train.values
    y_train_array = y_train.values
    X_test_array = X_test.values

    array_dict = {
        'X_Train': X_train_array,
        'y_Train': y_train_array,
        'X_Test': X_test_array
                 }
    return(array_dict)

In [ ]:
def Capture_Monthly_Seasonal_Effect(i_train_df):
    # i_train_df.loc[:, 'Seasonality_month'] = res_new.seasonal['month'].to_list()
    df = i_train_df[['date', 'Residual']]
    df.rename(columns = {'date':'ds', 'Residual':'y'}, inplace=True)
    df.insert(1, "unique_id", 1)
    
    models = [MSTL(
        season_length=[2922], # seasonalities of the time series 1) hourly * 4 2) monthly 365.25/12*24*4
        trend_forecaster=AutoARIMA() # model used to forecast trend
    )]
    
    sf = StatsForecast(
        models=models, # model used to fit each time series 
        freq='15min', # frequency of the data
    )

    sf = sf.fit(df=df)

    decomposition_df = sf.fitted_[0, 0].model_
    # print(decomposition_df)
    df.loc[:, 'MSTL_Trend'] = decomposition_df.loc[:, 'trend']
    df.loc[:, 'Monthly_Seasonality'] = decomposition_df.loc[:, 'seasonal']

    return(df)


In [ ]:
def MSTL_TS(i_train_df, i_validation_df, mstl_nr_freqs_to_keep):

    fmt = '%Y-%m-%d %H:%M:%S'
    d_max = dt.datetime.strptime(str(max(i_train_df['date'])), fmt)
    d_min = dt.datetime.strptime(str(min(i_train_df['date'])), fmt)
    
    week_diff = (d_max - d_min).days / 7

    if(week_diff > 52):
        mstl_seasonality_periods=["day_of_year", "month", "hour"] # , "day_of_week"
    else:
        mstl_seasonality_periods=["month", "hour"]
    
    i_ts_train_df = i_train_df.loc[:, ["date", target_col]]
    i_ts_train_df['date'] = pd.to_datetime(i_ts_train_df['date'])
    i_ts_train_df = i_ts_train_df.set_index("date")
    # i_ts_train = TimeSeries.from_series(i_ts_train_df)
    i_ts_train = i_ts_train_df[target_col]
    
    # MSTL decomposition of training data 
    stl = MultiSeasonalDecomposition(seasonal_model="fourier",
                                     seasonality_periods=mstl_seasonality_periods, 
                                     model = "additive", 
                                     n_fourier_terms=mstl_nr_freqs_to_keep)
    res_new = stl.fit(pd.Series(i_ts_train, index=i_ts_train_df.index))
    
    # adding decomposed components to df
    if(week_diff > 52):
        i_train_df.loc[:, 'Seasonality_Day_of_Year'] = res_new.seasonal['day_of_year'].to_list()
    i_train_df.loc[:, 'Seasonality_hour'] = res_new.seasonal['hour'].to_list()
    i_train_df.loc[:, 'Trend'] = res_new.trend.to_list()
    i_train_df.loc[:, 'Residual'] = res_new.resid.to_list()

    # the monthly component has not been captured properly. We are doing this using the MSTL model.
    df = Capture_Monthly_Seasonal_Effect(i_train_df)
    i_train_df.loc[:, 'Trend'] = df['MSTL_Trend'].to_list()
    i_train_df.loc[:, 'Seasonality_month'] = df['Monthly_Seasonality'].to_list()
    
    # Fitting FFT model to seasonal components and forecasting them for the test data.
    if(week_diff > 52):  
        i_sts_train = i_train_df.loc[:, ["date", 'Seasonality_Day_of_Year']].set_index("date")
        i_sts_train = TimeSeries.from_series(i_sts_train)
        fft_model = FFT(nr_freqs_to_keep=mstl_nr_freqs_to_keep, trend=None)
        fft_model.fit(i_sts_train)
        i_sts_validation = fft_model.predict(len(i_validation_df['date']))
        i_validation_df.loc[:, 'Seasonality_Day_of_Year'] = i_sts_validation.pd_series().to_list()
    
    i_sts_train = i_train_df.loc[:, ["date", 'Seasonality_month']].set_index("date")
    i_sts_train = TimeSeries.from_series(i_sts_train)
    fft_model = FFT(nr_freqs_to_keep=mstl_nr_freqs_to_keep, trend=None)
    fft_model.fit(i_sts_train)
    i_sts_validation = fft_model.predict(len(i_validation_df['date']))
    i_validation_df.loc[:, 'Seasonality_month'] = i_sts_validation.pd_series().to_list()
    
    i_sts_train = i_train_df.loc[:, ["date", 'Seasonality_hour']].set_index("date")
    i_sts_train = TimeSeries.from_series(i_sts_train)
    fft_model = FFT(nr_freqs_to_keep=mstl_nr_freqs_to_keep, trend=None)
    fft_model.fit(i_sts_train)
    i_sts_validation = fft_model.predict(len(i_validation_df['date']))
    i_validation_df.loc[:, 'Seasonality_hour'] = i_sts_validation.pd_series().to_list()
    
    # Exponential smoothing for trend component and forecasting into the future.
    i_tts_train = i_train_df.loc[:, ["date", 'Trend']].set_index("date")
    i_tts_train = TimeSeries.from_series(i_tts_train)
    es_model = ExponentialSmoothing(damped=True, 
                                    seasonal=None)
    es_model.fit(i_tts_train)
    i_tts_validation = es_model.predict(len(i_validation_df['date']))
    i_validation_df.loc[:, 'Trend'] = i_tts_validation.pd_series().to_list()

    
    if(week_diff > 52):
        i_ts_validation = (i_validation_df['Seasonality_Day_of_Year'] + 
                           i_validation_df['Seasonality_hour'] + 
                           i_validation_df['Seasonality_month'] + 
                           i_validation_df['Trend'])
        # ideally we should add the residual, but having some artificial error seems to help later on.
        i_fit = (i_train_df['Seasonality_Day_of_Year'] + 
                 i_train_df['Seasonality_hour'] + 
                 i_train_df['Seasonality_month'] + 
                 i_train_df['Trend'] - 
                 i_train_df['Residual'])
        # i_train_df.drop(columns=['Seasonality_Day_of_Year', 'Seasonality_hour', 'Seasonality_month', 'Trend', 'Residual'], inplace=True)
        # i_validation_df.drop(columns=['Seasonality_Day_of_Year', 'Seasonality_hour', 'Seasonality_month', 'Trend'], inplace=True)
    else:
        i_ts_validation = (i_validation_df['Seasonality_hour'] + 
                           i_validation_df['Seasonality_month'] + 
                           i_validation_df['Trend'])
        # ideally we should add the residual, but having some artificial error seems to help later on.
        i_fit = (i_train_df['Seasonality_hour'] + 
                 i_train_df['Seasonality_month'] + 
                 i_train_df['Trend'] - 
                 i_train_df['Residual'])
        
        # i_train_df.drop(columns=['Seasonality_hour', 'Seasonality_month', 'Trend', 'Residual'], inplace=True)
        # i_validation_df.drop(columns=['Seasonality_hour', 'Seasonality_month', 'Trend'], inplace=True)
    i_train_df.drop(columns=['Residual'], inplace=True)
    return(i_train_df, i_validation_df, i_fit, i_ts_validation)

In [ ]:
def TsPredAsFeature(train_df, test_df, target_col, windows_cv_num_periods, 
                    mstl_nr_freqs_to_keep = 35, fft_trend = None, fft_trend_poly_degree = 1):
    
    fmt = '%Y-%m-%d %H:%M:%S'
    d_max = dt.datetime.strptime(str(max(train_df['date'])), fmt)
    d_min = dt.datetime.strptime(str(min(train_df['date'])), fmt)
    
    time_diff = (d_max - d_min).total_seconds()
    period_time_diff = time_diff/windows_cv_num_periods
    
    train_period_end = []
    validation_period_end = []
    train_df.loc[:, 'TS_Pred'] = np.nan

    # generating out os sample TS values for training data
    for i in list(range(windows_cv_num_periods - 1)):
        
        train_period_end.append(d_min + dt.timedelta(seconds=period_time_diff * (i + 1)))
        validation_period_end.append(d_min + dt.timedelta(seconds=period_time_diff * (i + 2)))
    
        i_train_df = train_df[train_df['date'] <= train_period_end[i]]
        i_validation_df = train_df[((train_df['date'] >  train_period_end[i]) & 
                                    (train_df['date'] <= validation_period_end[i]))]
    
        # fft_model = FFT(nr_freqs_to_keep=mstl_nr_freqs_to_keep, trend=fft_trend, trend_poly_degree = fft_trend_poly_degree)
        # fft_model.fit(i_ts_train)
    
        # i_ts_validation = fft_model.predict(len(i_validation_df['date']))
        i_train_df, i_validation_df, i_fit, i_ts_validation = MSTL_TS(i_train_df, i_validation_df, mstl_nr_freqs_to_keep)
        
        if(i == 0):     
            mask = (train_df['date'] <= train_period_end[i])  
            train_df.loc[mask, 'TS_Pred'] = i_fit.to_list()
    
        mask = (train_df['date'] > train_period_end[i]) & (train_df['date'] <= validation_period_end[i])
        train_df.loc[mask, 'TS_Pred'] = i_ts_validation.to_list()

    train_df['TS_Pred'].fillna((train_df[target_col].mean()), inplace=True)

    train_df.loc[train_df['TS_Pred'].isnull(), 'TS_Pred'] = train_df[target_col].mean()
    
    # generating TS values for test data.
    train_df, test_df, i_fit, i_ts_validation = MSTL_TS(train_df, test_df, mstl_nr_freqs_to_keep)

    test_df.loc[:, 'TS_Pred'] = i_ts_validation.to_list()

    out_dict = {
        'train_df': train_df,
        'test_df' : test_df
    }
    return(out_dict)

In [ ]:
def FitModelAndPredict(train_df, X_train_array, y_train_array, test_df, X_test_array, new_quantiles, cb_param):

    train_dataset = cb.Pool(X_train_array, y_train_array) 

    new_quantiles_txt = ",".join(new_quantiles)
    
    model = cb.CatBoostRegressor(
        loss_function=''.join(['MultiQuantile:alpha=', new_quantiles_txt]), 
        # loss_function='Quantile:alpha=0.5', # CV on single quantile
        logging_level = 'Silent',
        random_seed = 42,
        depth = cb_param['depth'],
        iterations = cb_param['iterations'],
        l2_leaf_reg = cb_param['l2_leaf_reg'],
        learning_rate = cb_param['learning_rate'])

    # model.grid_search(grid, train_dataset)
    model.fit(train_dataset)

    pred = model.predict(X_train_array)
    pred_df = pd.DataFrame(pred, columns=new_quantiles)
    pred_df.loc[:, 'id']  = train_df["id"]
    new_cols = ['id'] + new_quantiles
    train_out = pred_df[new_cols]
    
    pred = model.predict(X_test_array)
    pred_df = pd.DataFrame(pred, columns=new_quantiles)
    pred_df.loc[:, 'id']  = test_df["id"]
    new_cols = ['id'] + new_quantiles
    test_out = pred_df[new_cols]
    
    out_dict = {
        'model': model,
        'train_out' : train_out, 
        'test_out' : test_out
    }
    return(out_dict)

### Validation

In [ ]:
train_val_df = train_df[train_df['date'] < evaluation_start_date]
test_val_df_t = train_df[train_df['date'] >= evaluation_start_date]
test_val_df_t.reset_index(inplace=True, drop = True)
test_val_df = test_val_df_t.drop(columns = target_col)  

In [ ]:
# Find right nr frequencies to keep.
# for darts-fft cross validation.
df1 = pd.DataFrame({'mstl_nr_freqs_to_keep': [35]}) # 1, 3, 5, 10, 25, 35, 50
df2 = pd.DataFrame({'fft_trend_poly_degree': [0], 'fft_trend': [None]})
df_ts_iter = df1.merge(df2, how="cross")

train_rmse_list = []
test_rmse_list = []
out_dict_list = []

for i in df_ts_iter.index:
    mstl_nr_freqs_to_keep = df_ts_iter.loc[i, 'mstl_nr_freqs_to_keep']
    fft_trend = df_ts_iter.loc[i, 'fft_trend']
    fft_trend_poly_degree = df_ts_iter.loc[i, 'fft_trend_poly_degree']
    
    out_dict = TsPredAsFeature(train_val_df, test_val_df, target_col, windows_cv_num_periods, 
                               mstl_nr_freqs_to_keep, fft_trend, fft_trend_poly_degree)
    train_val_df = out_dict["train_df"]
    test_val_df = out_dict["test_df"]

    out_dict_list.append(out_dict)
    train_rmse_list.append(root_mean_squared_error(train_val_df[target_col], train_val_df['TS_Pred']))
    test_rmse_list.append(root_mean_squared_error(test_val_df_t[target_col], test_val_df['TS_Pred']))

    print('-------------------------------------------------')
    print(f'mstl_nr_freqs_to_keep: {mstl_nr_freqs_to_keep}')
    print(f'fft_trend_poly_degree: {fft_trend_poly_degree}')
    print(f'train_rmse: {train_rmse_list[i]}')
    print(f'test_rmse: {test_rmse_list[i]}')
    print('-------------------------------------------------')

df_ts_iter['train_rmse'] = train_rmse_list
df_ts_iter['test_rmse'] = test_rmse_list

df_ts_iter['weighted_rmse'] = df_ts_iter['train_rmse'] * 0.1 + df_ts_iter['test_rmse'] * 0.9

In [ ]:
# selected time series hyper parameters
rmse_list = df_ts_iter['weighted_rmse'].to_list()
ind = rmse_list.index(min(rmse_list))
mstl_nr_freqs_to_keep = df_ts_iter.loc[ind, 'mstl_nr_freqs_to_keep']
fft_trend_poly_degree = df_ts_iter.loc[ind, 'fft_trend_poly_degree']
fft_trend = df_ts_iter.loc[ind, 'fft_trend']
out_dict = out_dict_list[ind]
train_val_df = out_dict["train_df"]
test_val_df = out_dict["test_df"]

train_val_df.drop(columns=['TS_Pred'], inplace=True)
test_val_df.drop(columns=['TS_Pred'], inplace=True)

print(f'index selected: {ind}')
print(f'mstl_nr_freqs_to_keep: {mstl_nr_freqs_to_keep}')
print(f'fft_trend_poly_degree: {fft_trend_poly_degree}')
print(f'fft_trend: {fft_trend}')

In [ ]:
### df to array
array_dict = ConvertDataFrameToArrays(train_val_df, test_val_df, target_col, tracking_col)
X_train_val_array = array_dict["X_Train"]
y_train_val_array = array_dict["y_Train"]
X_test_val_array = array_dict["X_Test"]

In [ ]:
# create data frame with hyper parameters of all combinations. 

df1 = pd.DataFrame({'iterations': [100, 200]}) # 150
df2 = pd.DataFrame({'learning_rate': [0.03, 0.1]})
df3 = pd.DataFrame({'depth': [2, 4]}) # 6, 8
df4 = pd.DataFrame({'l2_leaf_reg': [0.2, 3]}) # 0.5, 1

df_iter = df1.merge(df2, how="cross")
df_iter = df_iter.merge(df3, how="cross")
df_iter = df_iter.merge(df4, how="cross")

In [ ]:
## Testing for different hyper parameters.
v_rmse_list = []
v_crps_list = []
out_dict_list = []

for i in df_iter.index:

    cb_param = {'iterations': df_iter.loc[i, 'iterations'],
                'learning_rate': df_iter.loc[i, 'learning_rate'],
                'depth': df_iter.loc[i, 'depth'],
                'l2_leaf_reg': df_iter.loc[i, 'l2_leaf_reg']}

    out_dict = FitModelAndPredict(train_val_df, X_train_val_array, y_train_val_array, test_val_df, X_test_val_array,
                                  new_quantiles, cb_param)
    
    out_dict_list.append(out_dict)
    
    train_val_out = out_dict["train_out"]
    test_val_out = out_dict["test_out"]
    model_val = out_dict["model"]

    validation_rmse = root_mean_squared_error(test_val_df_t[target_col], test_val_out['0.5'])
    validation_crps = crps(test_val_out, test_val_df_t)

    v_rmse_list.append(validation_rmse)
    v_crps_list.append(validation_crps)
    
    print('-------------------------------------------------')
    print(f'index: {i}')
    print(cb_param)
    print(f'RMSE: {validation_rmse}')
    print(f'CRPS: {validation_crps}')
    print('-------------------------------------------------')

df_iter['RMSE'] = v_rmse_list
df_iter['CRPS'] = v_crps_list
df_iter['combined_score'] = df_iter['RMSE'] * 0.5 + df_iter['CRPS'] * 0.5

In [ ]:
# selected cb hyper parameters
rmse_list = df_iter['combined_score'].to_list()
ind = rmse_list.index(min(rmse_list))

cb_param = {'iterations': df_iter.loc[ind, 'iterations'],
            'learning_rate': df_iter.loc[ind, 'learning_rate'],
            'depth': df_iter.loc[ind, 'depth'],
            'l2_leaf_reg': df_iter.loc[ind, 'l2_leaf_reg']}

out_dict = out_dict_list[ind]
train_val_out = out_dict["train_out"]
test_val_out = out_dict["test_out"]
model_val = out_dict["model"]

print(f'index selected: {ind}')
print(f'cb_param: ')
print(df_iter.loc[ind, :])

In [ ]:
## Feature selection
a
v_rmse_list = []
v_crps_list = []
out_dict_list = []

for i in df_iter.index:

    

    out_dict = FitModelAndPredict(train_val_df, X_train_val_array, y_train_val_array, 
                                  test_val_df, X_test_val_array,
                                  new_quantiles, cb_param)
    
    out_dict_list.append(out_dict)
    
    train_val_out = out_dict["train_out"]
    test_val_out = out_dict["test_out"]
    model_val = out_dict["model"]

    validation_rmse = root_mean_squared_error(test_val_df_t[target_col], test_val_out['0.5'])
    validation_crps = crps(test_val_out, test_val_df_t)

    v_rmse_list.append(validation_rmse)
    v_crps_list.append(validation_crps)
    
    print('-------------------------------------------------')
    print(f'index: {i}')
    print(cb_param)
    print(f'RMSE: {validation_rmse}')
    print(f'CRPS: {validation_crps}')
    print('-------------------------------------------------')

df_iter['RMSE'] = v_rmse_list
df_iter['CRPS'] = v_crps_list
df_iter['combined_score'] = df_iter['RMSE'] * 0.5 + df_iter['CRPS'] * 0.5

In [ ]:
cols_to_rem = [target_col] + tracking_col
pd.DataFrame({'feature_importance': model_val.get_feature_importance(), 
              'feature_names': train_val_df.drop(columns=cols_to_rem).columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)

# model_val.get_feature_importance()
# model_val.feature_names_
# train_val_df.drop(columns=cols_to_rem).columns

In [ ]:
train_val_out.head(6)

In [ ]:
train_val_df.loc[:, 'pred'] = train_val_out['0.5']
train_val_df.loc[:, 'pred_UB'] = train_val_out['0.95']
train_val_df.loc[:, 'pred_LB'] = train_val_out['0.05']

test_val_df.loc[:, target_col] = test_val_df_t[target_col]
test_val_df.loc[:, 'pred'] = test_val_out['0.5']
test_val_df.loc[:, 'pred_UB'] = test_val_out['0.95']
test_val_df.loc[:, 'pred_LB'] = test_val_out['0.05']

In [ ]:
# plot TS vs Actuals train data
# melted_df = pd.melt(train_val_df, id_vars=['id', 'date'], 
#                     value_vars=['Temperature', 'TS_Pred'], 
#                     var_name='variable', value_name='Temp')
# fig = px.line(melted_df, x="date", y="Temp", color = 'variable', title="Temperature - Actuals vs TS (FFT) - Train")
# fig.show()

In [ ]:
# plot TS vs Actuals test data
# melted_df = pd.melt(test_val_df, id_vars=['id', 'date'], 
#                     value_vars=['Temperature', 'TS_Pred'], 
#                     var_name='variable', value_name='Temp')
# fig = px.line(melted_df, x="date", y="Temp", color = 'variable', title="Temperature - Actuals vs TS (FFT) - Test")
# fig.show()

In [ ]:
# plot train data
melted_df = pd.melt(train_val_df, id_vars=['id', 'date'], 
                    value_vars=['Temperature', 'pred', 'pred_UB', 'pred_LB'], 
                    var_name='variable', value_name='Temp')
fig = px.line(melted_df, x="date", y="Temp", color = 'variable', title="Temperature over time - train")
fig.show()

In [ ]:
# plot validation data
melted_df = pd.melt(test_val_df, id_vars=['id', 'date'], 
                    value_vars=['Temperature', 'pred', 'pred_UB', 'pred_LB'], 
                    var_name='variable', value_name='Temp')
fig = px.line(melted_df, x="date", y="Temp", color = 'variable', title="Temperature over time - test")
fig.show()

### Training using full data

In [ ]:
out_dict = TsPredAsFeature(train_df, test_df, target_col, windows_cv_num_periods, 
                           mstl_nr_freqs_to_keep, fft_trend, fft_trend_poly_degree)
train_df = out_dict["train_df"]
test_df = out_dict["test_df"]

train_df.drop(columns=['TS_Pred'], inplace=True)
test_df.drop(columns=['TS_Pred'], inplace=True)

In [ ]:
train_df.head()

In [ ]:
### df to array
array_dict = ConvertDataFrameToArrays(train_df, test_df, target_col, tracking_col)

# Convert DataFrame to numpy arrays
X_train_array = array_dict["X_Train"]
y_train_array = array_dict["y_Train"]
X_test_array = array_dict["X_Test"]

In [ ]:
# fitting model and getting predictions
out_dict = FitModelAndPredict(train_df, X_train_array, y_train_array, test_df, X_test_array,
                              new_quantiles, cb_param)
train_out = out_dict["train_out"]
test_out = out_dict["test_out"]

In [ ]:
train_out.head()

In [ ]:
test_out.to_csv(f'outputs/test_submission_exp{exp_no}.csv', index = False)

### Plotting the output

In [ ]:
# plot the result
train_df['pred'] = train_out['0.5']
train_df['pred_UB'] = train_out['0.95']
train_df['pred_LB'] = train_out['0.05']

test_df['pred'] = test_out['0.5']
test_df['pred_UB'] = test_out['0.95']
test_df['pred_LB'] = test_out['0.05']

In [ ]:
train_sub_df = train_df[['id', 'date', 'Temperature', 'pred', 'pred_UB', 'pred_LB']]
test_sub_df = test_df[['id', 'date', 'pred', 'pred_UB', 'pred_LB']]
plot_df = pd.concat([train_sub_df, test_sub_df], axis = 0, ignore_index = True)

In [ ]:
# Melt the DataFrame
melted_df = pd.melt(plot_df, id_vars=['id', 'date'], 
                    value_vars=['Temperature', 'pred', 'pred_UB', 'pred_LB'], 
                    var_name='variable', value_name='Temp')

In [ ]:
fig = px.line(melted_df, x="date", y="Temp", color = 'variable', title="Temperature over time")
fig.show()